In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Setup

In [1]:
!pip install transformers datasets pytorch-crf --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.5 MB/s eta 0:00:00


In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torchcrf import CRF

from datasets import Dataset, DatasetDict, load_dataset
import random

import transformers
from transformers import BertTokenizerFast, DistilBertModel, DistilBertConfig
from transformers import BertModel, AdamW, get_linear_schedule_with_warmup

from torch.optim import AdamW

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

## Dataset

In [ ]:
def tokenize_and_align_labels(examples, tokenizer):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        max_length=180, truncation=True, padding="max_length",
        is_split_into_words=True, add_special_tokens=False,
        return_tensors = "pt",
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)

        label = []
        for i in range(len(word_ids)):
          if word_ids[i] == None:
            label.append(0)
          elif labels[word_ids[i]] % 2 == 1 and word_ids[i-1] == word_ids[i]:
            label.append(labels[word_ids[i]] + 1)
          else:
            label.append(labels[word_ids[i]])
        new_labels.append(label)

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
label_to_tag = {
    'O':0,
    'B-Drug':1, 'I-Drug':2,
    'B-Reason':3, 'I-Reason':4,
    'B-Route':5, 'I-Route':6,
    'B-Strength':7, 'I-Strength':8,
    'B-Form':9, 'I-Form':10,
    'B-Dosage':11, 'I-Dosage':12,
    'B-Frequency':13, 'I-Frequency':14,
    'B-Duration':15, 'I-Duration':16,
    'B-ADE':17, 'I-ADE':18,
}

nb_unique_tags = len(label_to_tag)

In [ ]:
def get_dataset(TOKENIZER_NAME):
  tokenizer = BertTokenizerFast.from_pretrained(TOKENIZER_NAME)

  data = {
    "train": {"tokens": [], "labels": [], "ner_tags": []},
    "valid": {"tokens": [], "labels": [], "ner_tags": []},
    "test": {"tokens": [], "labels": [], "ner_tags": []}
  }

  def populate_dataset(file_path, train_valid_or_test):
    with open(file_path) as f:
      tokens, labels, ner_tags = [], [], []
      for line in f.read().splitlines():
        if len(line) < 2:
          is_taken = train_valid_or_test != "train" or ner_tags.count(0) != len(ner_tags) or  random.randint(1, 100) > 92 # 25% of skipping sample if no-label (other than 0)
          if len(tokens) != 0 and is_taken: # bypass double line breaks
            data[train_valid_or_test]['tokens'].append(tokens)
            data[train_valid_or_test]['labels'].append(labels)
            data[train_valid_or_test]['ner_tags'].append(ner_tags)
          tokens, labels, ner_tags = [], [], []
          continue
        line = line.split(' ')
        token, label, tag = ' '.join(line[:-4]), line[-1], label_to_tag[line[-1]]
        tokens.append(token)
        labels.append(label)
        ner_tags.append(tag)


  populate_dataset(file_path='drive/MyDrive/TransformerCRF-v2/Dataset/train_spacy.txt', train_valid_or_test='train')
  populate_dataset(file_path='drive/MyDrive/TransformerCRF-v2/Dataset/valid_spacy.txt', train_valid_or_test='valid')
  populate_dataset(file_path='drive/MyDrive/TransformerCRF-v2/Dataset/test_spacy.txt', train_valid_or_test='test')

  train_dataset = Dataset.from_dict(data["train"]).shuffle()
  valid_dataset = Dataset.from_dict(data["valid"]).shuffle()
  test_dataset = Dataset.from_dict(data["test"]).shuffle()

  dataset = DatasetDict({
    "train": train_dataset,
    "validation": valid_dataset,
    "test": test_dataset
  })

  dataset = dataset.map(
      lambda x: tokenize_and_align_labels(x, tokenizer),
      batched=True,
      remove_columns=dataset["train"].column_names,
  ).with_format('torch')

  return dataset

14041 -> no-label: 2909 (20.72%)

41497 -> no-label: 31163 (75.10%) (2909 no-label reached at 3829th sample -> at this stage it represented 75.97%)

We want to only keep N no-label so we reduce to X% (we tos a coin with Y% chance to keep a no-label sample)

Now we get (on average):

In [ ]:
dataset = get_dataset(TOKENIZER_NAME = 'bert-base-cased')

Map:   0%|          | 0/12782 [00:00<?, ? examples/s]

Map:   0%|          | 0/4536 [00:00<?, ? examples/s]

Map:   0%|          | 0/30614 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 12782
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4536
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 30614
    })
})

## Model

In [ ]:
import os
from typing import List, Tuple

import torch
from torch import nn
from transformers import AutoModel

LOG_INF = 10e5

class BertCRF(nn.Module):
    def __init__(self, num_labels: int, use_crf, vocab_size):
        super().__init__()
        self.num_labels = num_labels
        self.use_crf = use_crf
        self.cross_entropy = nn.CrossEntropyLoss()

        # PLM
        # self.bert = AutoModel.from_pretrained(MODEL_NAME)
        self.bert = DistilBertModel(config=DistilBertConfig(vocab_size=vocab_size))

        # Classification/Adpation Layer
        self.dropout = nn.Dropout(0.2)
        self.hidden2label = nn.Linear(self.bert.config.hidden_size, num_labels)

        # CRF Layer
        self.start_transitions = nn.Parameter(torch.empty(num_labels))
        self.end_transitions = nn.Parameter(torch.empty(num_labels))
        self.transitions = nn.Parameter(torch.empty(num_labels, num_labels))

        nn.init.uniform_(self.start_transitions, -0.1, 0.1)
        nn.init.uniform_(self.end_transitions, -0.1, 0.1)
        nn.init.uniform_(self.transitions, -0.1, 0.1)

    def _compute_log_denominator(self, features: torch.Tensor, mask: torch.Tensor) -> torch.Tensor:
        seq_len = features.shape[0]

        log_score_over_all_seq = self.start_transitions + features[0]

        for i in range(1, seq_len):
            next_log_score_over_all_seq = torch.logsumexp(
                log_score_over_all_seq.unsqueeze(2) + self.transitions + features[i].unsqueeze(1),
                dim=1,
            )
            log_score_over_all_seq = torch.where(
                mask[i].unsqueeze(1),
                next_log_score_over_all_seq,
                log_score_over_all_seq,
            )
        log_score_over_all_seq += self.end_transitions
        return torch.logsumexp(log_score_over_all_seq, dim=1)

    def _compute_log_numerator(self, features: torch.Tensor, labels: torch.Tensor, mask: torch.Tensor) -> torch.Tensor:
        seq_len, bs, _ = features.shape

        score_over_seq = self.start_transitions[labels[0]] + features[0, torch.arange(bs), labels[0]]

        for i in range(1, seq_len):
            score_over_seq += (
                self.transitions[labels[i - 1], labels[i]] + features[i, torch.arange(bs), labels[i]]
            ) * mask[i]
        seq_lens = mask.sum(dim=0) - 1
        last_tags = labels[seq_lens.long(), torch.arange(bs)]
        score_over_seq += self.end_transitions[last_tags]
        return score_over_seq

    def get_bert_features(self, input_ids: torch.Tensor, attention_mask: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        hidden = self.bert(input_ids, attention_mask=attention_mask)["last_hidden_state"]
        hidden = self.dropout(hidden)
        return self.hidden2label(hidden), hidden

    def forward(self, input_ids: torch.Tensor, attention_mask: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
        features, _ = self.get_bert_features(input_ids=input_ids, attention_mask=attention_mask)
        attention_mask = attention_mask.bool()

        if self.use_crf:
            features = torch.swapaxes(features, 0, 1)
            attention_mask = torch.swapaxes(attention_mask, 0, 1)
            labels = torch.swapaxes(labels, 0, 1)

            log_numerator = self._compute_log_numerator(features=features, labels=labels, mask=attention_mask)
            log_denominator = self._compute_log_denominator(features=features, mask=attention_mask)

            return torch.mean(log_denominator - log_numerator)
        else:
            return self.cross_entropy(
                features.flatten(end_dim=1),
                torch.where(attention_mask.bool(), labels, -100).flatten(end_dim=1),
            )

    def _viterbi_decode(self, features: torch.Tensor, mask: torch.Tensor) -> List[List[int]]:
        seq_len, bs, _ = features.shape

        log_score_over_all_seq = self.start_transitions + features[0]

        backpointers = torch.empty_like(features)

        for i in range(1, seq_len):
            next_log_score_over_all_seq = (
                log_score_over_all_seq.unsqueeze(2) + self.transitions + features[i].unsqueeze(1)
            )

            next_log_score_over_all_seq, indices = next_log_score_over_all_seq.max(dim=1)

            log_score_over_all_seq = torch.where(
                mask[i].unsqueeze(1),
                next_log_score_over_all_seq,
                log_score_over_all_seq,
            )
            backpointers[i] = indices

        backpointers = backpointers[1:].int()

        log_score_over_all_seq += self.end_transitions
        seq_lens = mask.sum(dim=0) - 1

        best_paths = []
        for seq_ind in range(bs):
            best_label_id = torch.argmax(log_score_over_all_seq[seq_ind]).item()
            best_path = [best_label_id]

            for backpointer in reversed(backpointers[: seq_lens[seq_ind]]):
                best_path.append(backpointer[seq_ind][best_path[-1]].item())

            best_path.reverse()
            best_paths.append(best_path)

        return best_paths

    def decode(self, input_ids: torch.Tensor, attention_mask: torch.Tensor) -> List[List[int]]:
        features, _ = self.get_bert_features(input_ids=input_ids, attention_mask=attention_mask)
        attention_mask = attention_mask.bool()

        if self.use_crf:
            features = torch.swapaxes(features, 0, 1)
            mask = torch.swapaxes(attention_mask, 0, 1)
            predictions = self._viterbi_decode(features=features, mask=mask)
            # Apply padding
            return torch.tensor([p + [0]*(len(attention_mask[-1]) - len(p)) for p in predictions])
        else:
            labels = torch.argmax(features, dim=2)
            predictions = []
            for i in range(len(labels)):
                predictions.append(labels[i][attention_mask[i]].tolist())
            # Apply padding
            return torch.tensor([p + [0]*(len(attention_mask[-1]) - len(p)) for p in predictions])

## Train function

In [ ]:
def calculate_metrics(predicted_labels, true_labels, masking):

  predicted_labels = predicted_labels.cpu().numpy()
  true_labels = true_labels.cpu().numpy()
  masking = masking.cpu().numpy()

  # Flatten the arrays to 1D arrays
  predicted_labels_flat = predicted_labels.flatten()
  true_labels_flat = true_labels.flatten()
  masking = masking.flatten()

  # Use valid indices to filter out -100 tokens
  predicted_labels_flat = predicted_labels_flat[masking]
  true_labels_flat = true_labels_flat[masking]

  # Calculate accuracy, precision, recall, and F1-score for each class
  accuracy = accuracy_score(true_labels_flat, predicted_labels_flat)

  macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(true_labels_flat, predicted_labels_flat, average='macro')
  weight_precision, weight_recall, weight_f1, _ = precision_recall_fscore_support(true_labels_flat, predicted_labels_flat, average='weighted')

  metrics_report = classification_report(true_labels_flat, predicted_labels_flat)

  # Print or use the calculated metrics
  print(f"Accuracy: {accuracy:.4f}")
  print(f"WEIGHTED -> Precision:{weight_precision:.4f}, Recall:{weight_recall:.4f}, F1:{weight_f1:.4f}.")
  print(f"MACRO    -> Precision:{macro_precision:.4f}, Recall:{macro_recall:.4f}, F1:{macro_f1:.4f}.")
  print()
  print("Metrics Report:", metrics_report)
  # plt.show() # print("Confusion Matrix:", confusion_matrix)

  return (macro_precision, macro_recall, macro_f1), (weight_precision, weight_recall, weight_f1)


In [ ]:
def train(model, train_dataloader, eval_dataloader, nb_epochs, optimizer, scheduler, device, RUN_NAME):

  epoch_metrics_and_loss = []
  for epoch in range(nb_epochs):
    print("Epoch", epoch+1)

    # Training
    model.train()
    epoch_loss = []
    for batch in train_dataloader:

      optimizer.zero_grad()

      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)

      loss = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

      loss.backward()
      epoch_loss.append(loss.item())

      optimizer.step()
      scheduler.step()

    training_loss = np.mean(epoch_loss)
    print("Training Loss:", training_loss)

    # Evaluation
    model.eval()
    epoch_loss = []
    all_predictions, all_labels, all_masks = torch.tensor([], device=device), torch.tensor([], device=device), torch.tensor([], device=device)
    for batch in eval_dataloader:

      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)

      with torch.no_grad():
        predictions = model.decode(input_ids=input_ids, attention_mask=attention_mask)
        loss = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        epoch_loss.append(loss.item())

      all_predictions = torch.cat((all_predictions, predictions.to(device)), dim=0)
      all_labels = torch.cat((all_labels, labels.to(device)), dim=0)
      all_masks = torch.cat((all_masks, attention_mask), dim=0)

    eval_loss = np.mean(epoch_loss)
    print("Evaluation Loss:", eval_loss)
    macros, weighteds = calculate_metrics(all_predictions, all_labels, all_masks.bool())
    print()

    with open('drive/MyDrive/TransformerCRF-v2/night_scores_2.txt', 'a') as f:
      f.write(f'{RUN_NAME}\n')
      f.write(f'Epoch {epoch+1}\n')
      f.write(f'Training Loss: {round(training_loss, 4)}, Evaluation Loss: {round(eval_loss, 4)}\n')
      f.write(f'Macro Scores (p/r/f1): {"/".join([str(round(s, 4)) for s in macros])}\n')
      f.write(f'Weighted Scores (p/r/f1): {"/".join([str(round(s, 4)) for s in weighteds])}\n\n')


## Hyperparameter selection pipeline

In [ ]:
hyperparameters = [
    # Model_name, Batch_size, Learning rate, Uses CRF Layer (bool)
    # ('bert-base-cased', 14, 1e-4, True),
    # ('bert-base-cased', 14, 5e-5, True),
    # ('bert-base-cased', 8, 1e-4, True),
    # ('bert-base-cased', 8, 5e-5, True),

    # ('bert-base-cased', 14, 1e-4, False),
    # ('bert-base-cased', 14, 5e-5, False),
    # ('bert-base-cased', 8, 1e-4, False),
    # ('bert-base-cased', 8, 5e-5, False),

    # ('dmis-lab/biobert-base-cased-v1.2', 14, 1e-4, True),
    # ('dmis-lab/biobert-base-cased-v1.2', 14, 5e-5, True),
    # ('dmis-lab/biobert-base-cased-v1.2', 8, 1e-4, True),
    # ('dmis-lab/biobert-base-cased-v1.2', 8, 5e-5, True),

    # ('dmis-lab/biobert-base-cased-v1.2', 14, 1e-4, False),
    # ('dmis-lab/biobert-base-cased-v1.2', 14, 5e-5, False),
    # ('dmis-lab/biobert-base-cased-v1.2', 8, 1e-4, False),
    # ('dmis-lab/biobert-base-cased-v1.2', 8, 5e-5, False),

    # ('medicalai/ClinicalBERT', 14, 1e-4, True),
    # ('medicalai/ClinicalBERT', 14, 5e-5, True),
    # ('medicalai/ClinicalBERT', 8, 1e-4, True),
    # ('medicalai/ClinicalBERT', 8, 5e-5, True),

    # ('medicalai/ClinicalBERT', 14, 1e-4, False),
    # ('medicalai/ClinicalBERT', 14, 5e-5, False),
    # ('medicalai/ClinicalBERT', 8, 1e-4, False),
    # ('medicalai/ClinicalBERT', 8, 5e-5, False),

    ('TransformerCRF-distilbert-base-cased', 8, 5e-5, False),
    ('TransformerCRF-distilbert-base-cased', 8, 1e-4, False),
    ('TransformerCRF-distilbert-base-cased', 8, 5e-4, False),
    ('TransformerCRF-distilbert-base-cased', 14, 5e-5, False),
    ('TransformerCRF-distilbert-base-cased', 14, 1e-4, False),
    ('TransformerCRF-distilbert-base-cased', 14, 5e-4, False),

    ('TransformerCRF-distilbert-base-cased', 8, 5e-5, True),
    ('TransformerCRF-distilbert-base-cased', 8, 1e-4, True),
    ('TransformerCRF-distilbert-base-cased', 8, 5e-4, True),
    ('TransformerCRF-distilbert-base-cased', 14, 5e-5, True),
    ('TransformerCRF-distilbert-base-cased', 14, 1e-4, True),
    ('TransformerCRF-distilbert-base-cased', 14, 5e-4, True),
]

In [ ]:
datasets = {
    'TransformerCRF-distilbert-base-cased': get_dataset(TOKENIZER_NAME = 'distilbert-base-cased'),
    # 'bert-base-cased': get_dataset(TOKENIZER_NAME = 'bert-base-cased'),
    # 'dmis-lab/biobert-base-cased-v1.2': get_dataset(TOKENIZER_NAME = 'dmis-lab/biobert-base-cased-v1.2'),
    # 'medicalai/ClinicalBERT': get_dataset(TOKENIZER_NAME = 'medicalai/ClinicalBERT'),
}

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


Map:   0%|          | 0/12832 [00:00<?, ? examples/s]

Map:   0%|          | 0/4536 [00:00<?, ? examples/s]

Map:   0%|          | 0/30614 [00:00<?, ? examples/s]

In [ ]:
for hyperparam in hyperparameters:
  MODEL_NAME, BATCH_SIZE, LEARNING_RATE, USES_CRF = hyperparam
  NUM_EPOCHS = 6
  RUN_NAME = f"{MODEL_NAME.replace('/','_')} - {BATCH_SIZE} - {LEARNING_RATE} - {USES_CRF}"

  dataset = datasets[MODEL_NAME]

  train_dataloader = DataLoader(dataset['train'], batch_size=BATCH_SIZE, shuffle=True) # Training Dataset is (hopefully?) already shuffled in Dataset Extraction fct
  eval_dataloader = DataLoader(dataset['validation'], batch_size=BATCH_SIZE, shuffle=False)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  model = BertCRF(num_labels=nb_unique_tags, use_crf=USES_CRF, vocab_size=len(BertTokenizerFast.from_pretrained('distilbert-base-cased').vocab)).to(device)

  optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
  total_steps = len(train_dataloader) * NUM_EPOCHS
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

  # Train
  train(model, train_dataloader, eval_dataloader, NUM_EPOCHS, optimizer, scheduler, device, RUN_NAME)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


Epoch 1
Training Loss: 0.5721047202083387
Evaluation Loss: 0.2825152178211187
Accuracy: 0.9231
WEIGHTED -> Precision:0.9337, Recall:0.9231, F1:0.9263.
MACRO    -> Precision:0.6278, Recall:0.6509, F1:0.6199.

Metrics Report:               precision    recall  f1-score   support

         0.0       0.98      0.94      0.96    103231
         1.0       0.59      0.81      0.68      1736
         2.0       0.66      0.91      0.77      5557
         3.0       0.37      0.35      0.36       457
         4.0       0.25      0.33      0.29      1063
         5.0       0.74      0.91      0.82       556
         6.0       0.87      0.91      0.89       565
         7.0       0.78      0.86      0.82       676
         8.0       0.79      0.88      0.83      1267
         9.0       0.83      0.90      0.86       661
        10.0       0.76      0.91      0.83      1332
        11.0       0.67      0.75      0.71       476
        12.0       0.85      0.82      0.83       911
        13.0       

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


Epoch 1
Training Loss: 0.5217584115720152
Evaluation Loss: 0.1834335653489124
Accuracy: 0.9500
WEIGHTED -> Precision:0.9464, Recall:0.9500, F1:0.9461.
MACRO    -> Precision:0.7697, Recall:0.6572, F1:0.6783.

Metrics Report:               precision    recall  f1-score   support

         0.0       0.97      0.98      0.98    103231
         1.0       0.76      0.81      0.78      1736
         2.0       0.85      0.81      0.83      5557
         3.0       0.67      0.30      0.41       457
         4.0       0.67      0.21      0.32      1063
         5.0       0.88      0.92      0.90       556
         6.0       0.94      0.90      0.92       565
         7.0       0.87      0.91      0.89       676
         8.0       0.91      0.86      0.88      1267
         9.0       0.85      0.93      0.89       661
        10.0       0.82      0.90      0.86      1332
        11.0       0.90      0.72      0.80       476
        12.0       0.89      0.88      0.88       911
        13.0       

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


...

Epoch 1
Training Loss: 19.56074439244286
Evaluation Loss: 6.332709903334394
Accuracy: 0.9318
WEIGHTED -> Precision:0.9303, Recall:0.9318, F1:0.9291.
MACRO    -> Precision:0.6689, Recall:0.6348, F1:0.6293.

Metrics Report:               precision    recall  f1-score   support

         0.0       0.97      0.97      0.97    103231
         1.0       0.59      0.78      0.67      1736
         2.0       0.75      0.73      0.74      5557
         3.0       0.57      0.28      0.38       457
         4.0       0.62      0.18      0.28      1063
         5.0       0.75      0.92      0.82       556
         6.0       0.88      0.91      0.89       565
         7.0       0.78      0.87      0.82       676
         8.0       0.85      0.79      0.82      1267
         9.0       0.86      0.90      0.88       661
        10.0       0.76      0.90      0.82      1332
        11.0       0.85      0.68      0.76       476
        12.0       0.83      0.81      0.82       911
        13.0       0.

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


Epoch 1
Training Loss: 17.302997110454182
Evaluation Loss: 6.434139759452255
Accuracy: 0.9436
WEIGHTED -> Precision:0.9435, Recall:0.9436, F1:0.9420.
MACRO    -> Precision:0.7128, Recall:0.6740, F1:0.6775.

Metrics Report:               precision    recall  f1-score   support

         0.0       0.98      0.97      0.97    103231
         1.0       0.74      0.81      0.77      1736
         2.0       0.71      0.92      0.81      5557
         3.0       0.57      0.37      0.45       457
         4.0       0.54      0.28      0.37      1063
         5.0       0.81      0.93      0.87       556
         6.0       0.88      0.92      0.90       565
         7.0       0.86      0.90      0.88       676
         8.0       0.87      0.91      0.89      1267
         9.0       0.89      0.90      0.89       661
        10.0       0.83      0.86      0.85      1332
        11.0       0.89      0.74      0.81       476
        12.0       0.88      0.87      0.87       911
        13.0       0

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


Epoch 1
Training Loss: 48.82475115818702
Evaluation Loss: 18.6738739322733
Accuracy: 0.8485
WEIGHTED -> Precision:0.7200, Recall:0.8485, F1:0.7790.
MACRO    -> Precision:0.0447, Recall:0.0526, F1:0.0483.

Metrics Report:               precision    recall  f1-score   support

         0.0       0.85      1.00      0.92    103231
         1.0       0.00      0.00      0.00      1736
         2.0       0.00      0.00      0.00      5557
         3.0       0.00      0.00      0.00       457
         4.0       0.00      0.00      0.00      1063
         5.0       0.00      0.00      0.00       556
         6.0       0.00      0.00      0.00       565
         7.0       0.00      0.00      0.00       676
         8.0       0.00      0.00      0.00      1267
         9.0       0.00      0.00      0.00       661
        10.0       0.00      0.00      0.00      1332
        11.0       0.00      0.00      0.00       476
        12.0       0.00      0.00      0.00       911
        13.0       0.0